<a href="https://colab.research.google.com/github/i-ganza007/PacMan_Formative/blob/main/CNNPOLICY_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y gymnasium stable-baselines3 ale-py shimmy tensorboard protobuf -y
!rm -rf /usr/local/lib/python3.11/dist-packages/gymnasium*
!rm -rf ~/.cache/pip
!pip cache purge

!pip install --no-cache-dir \
    "protobuf==3.20.3" \
    "tensorboard==2.14.0" \
    "gymnasium[atari,accept-rom-license]==0.29.1" \
    "ale-py==0.8.1" \
    "shimmy==0.2.1" \
    "stable-baselines3==2.3.0" \
    "autorom[accept-rom-license]"

!python -c "import gymnasium.vector; print(dir(gymnasium.vector))"

In [ ]:
import os
import gc
import torch
import warnings
import numpy as np
import gymnasium as gym
import ale_py
from datetime import datetime
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
warnings.filterwarnings('ignore')
gym.register_envs(ale_py)
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CallbackList
from gymnasium.wrappers import GrayScaleObservation, ResizeObservation, TransformReward

torch.cuda.empty_cache()
gc.collect()

CONFIG_ID = 1
configs = [
    {"lr": 2e-4, "gamma": 0.999, "batch": 32,
     "epsilon_start": 1.0, "epsilon_end": 0.05, "epsilon_decay": 0.3,
     "buffer": 20_000, "img_size": 64, "stack": 1, "net_arch": [128, 128]},

    {"lr": 1e-4, "gamma": 0.99, "batch": 32,
     "epsilon_start": 1.0, "epsilon_end": 0.02, "epsilon_decay": 0.25,
     "buffer": 20_000, "img_size": 64, "stack": 1, "net_arch": [128, 128]},

    {"lr": 3e-4, "gamma": 0.999, "batch": 64,
     "epsilon_start": 1.0, "epsilon_end": 0.1, "epsilon_decay": 0.2,
     "buffer": 25_000, "img_size": 64, "stack": 1, "net_arch": [128, 128]},

    {"lr": 2e-4, "gamma": 0.95, "batch": 16,
     "epsilon_start": 1.0, "epsilon_end": 0.01, "epsilon_decay": 0.4,
     "buffer": 15_000, "img_size": 64, "stack": 1, "net_arch": [64, 64]},

    {"lr": 1.5e-4, "gamma": 0.995, "batch": 32,
     "epsilon_start": 1.0, "epsilon_end": 0.05, "epsilon_decay": 0.25,
     "buffer": 20_000, "img_size": 64, "stack": 1, "net_arch": [128, 128]},
]

cfg = configs[CONFIG_ID - 1]

In [ ]:
print(f" PACMAN DQN TRAINING - CONFIG {CONFIG_ID}")

for k, v in cfg.items():
    if k == "epsilon_decay":
        print(f"{k:18}: {v:.1%} of timesteps")
    elif k == "net_arch":
        print(f"{k:18}: {v}")
    else:
        print(f"{k:18}: {v}")
print(f"{'GPU':18}: {torch.cuda.get_device_name(0)}")
print(f"{'VRAM Total':18}: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


 PACMAN DQN TRAINING - CONFIG 1
lr                : 0.0002
gamma             : 0.999
batch             : 32
epsilon_start     : 1.0
epsilon_end       : 0.05
epsilon_decay     : 30.0% of timesteps
buffer            : 20000
img_size          : 64
stack             : 1
net_arch          : [128, 128]
GPU               : Tesla P100-PCIE-16GB
VRAM Total        : 17.1 GB


In [ ]:
def make_base_env(render_mode=None):
    env = gym.make("ALE/Pacman-v5", frameskip=4, render_mode=render_mode)
    env = GrayScaleObservation(env)
    env = ResizeObservation(env, (cfg["img_size"], cfg["img_size"]))
    env = TransformReward(env, lambda r: np.clip(r, -1.0, 1.0))
    return env

train_raw = make_base_env()
train_raw = Monitor(train_raw, filename=f"./logs/episode_logs/cfg{CONFIG_ID}.monitor.csv", allow_early_resets=True)
train_env = DummyVecEnv([lambda: train_raw])
if cfg["stack"] > 1:
    train_env = VecFrameStack(train_env, n_stack=cfg["stack"])

eval_raw = make_base_env()
eval_env = DummyVecEnv([lambda: eval_raw])
if cfg["stack"] > 1:
    eval_env = VecFrameStack(eval_env, n_stack=cfg["stack"])

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [ ]:
model = DQN(
    "MlpPolicy",
    train_env,
    learning_rate=cfg["lr"],
    gamma=cfg["gamma"],
    batch_size=cfg["batch"],
    buffer_size=cfg["buffer"],
    learning_starts=5_000,
    target_update_interval=1_000,
    train_freq=4,
    gradient_steps=1,
    exploration_initial_eps=cfg["epsilon_start"],
    exploration_final_eps=cfg["epsilon_end"],
    exploration_fraction=cfg["epsilon_decay"],
    policy_kwargs=dict(net_arch=cfg["net_arch"]),
    tensorboard_log="./logs/tensorboard/",
    device="cuda",
    verbose=1
)

print(f"Initial VRAM Usage: {torch.cuda.memory_allocated() / 1e9:.2f} GB\n")

Using cuda device
Initial VRAM Usage: 0.00 GB



In [ ]:
class MemoryCallback(BaseCallback):
    def __init__(self, check_freq=10_000):
        super().__init__()
        self.check_freq = check_freq
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            alloc = torch.cuda.memory_allocated() / 1e9
            reserved = torch.cuda.memory_reserved() / 1e9
            peak = torch.cuda.max_memory_allocated() / 1e9
            print(f"Step {self.n_calls:,} | VRAM: {alloc:.2f}G alloc | {reserved:.2f}G res | {peak:.2f}G peak")
        return True

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=f"./logs/best_models/cfg{CONFIG_ID}/",
    log_path=f"./logs/eval_logs/cfg{CONFIG_ID}/",
    eval_freq=10_000,
    n_eval_episodes=5,
    deterministic=False,
    render=False,
    verbose=1
)

callback = CallbackList([MemoryCallback(check_freq=10_000), eval_callback])

total_timesteps = 1_500_000
print(f"Starting training for {total_timesteps:,} timesteps...")
print(" Logs:")
print(f" • Episode CSV → ./logs/episode_logs/cfg{CONFIG_ID}.monitor.csv")
print(f" • TensorBoard → tensorboard --logdir ./logs/tensorboard/")
print(f" • Eval Results → ./logs/eval_logs/cfg{CONFIG_ID}/")
print(f" • Best Model → ./logs/best_models/cfg{CONFIG_ID}/\n")

model.learn(
    total_timesteps=total_timesteps,
    tb_log_name=f"cfg{CONFIG_ID}_p100",
    log_interval=1,
    callback=callback,
    progress_bar=True,
    reset_num_timesteps=True
)

print("\nEvaluating final policy...")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=15, deterministic=False)  # ← FIXED

model.save(f"dqn_pacman_p100_cfg{CONFIG_ID}")
peak_vram = torch.cuda.max_memory_allocated() / 1e9

print(" TRAINING COMPLETE")
print(f" Config ID : {CONFIG_ID}")
print(f" Mean Reward : {mean_reward:.1f} ± {std_reward:.1f}")
print(f" Peak VRAM : {peak_vram:.2f} GB")
print(f" Model Saved : dqn_pacman_p100_cfg{CONFIG_ID}.zip")
print(f" TensorBoard : tensorboard --logdir ./logs/tensorboard/")

print("\nStarting LIVE RENDER – Close window to stop...")
render_env = make_base_env(render_mode="human")
obs, _ = render_env.reset()
for _ in range(2000):
    action, _ = model.predict(obs, deterministic=False)
    obs, reward, terminated, truncated, info = render_env.step(action)
    render_env.render()
    if terminated or truncated:
        obs, _ = render_env.reset()
render_env.close()

torch.cuda.empty_cache()
gc.collect()
train_env.close()
eval_env.close()
print("\nCleanup complete. Ready for next run!\n")

Starting training for 1,500,000 timesteps...
 Logs:
 • Episode CSV → ./logs/episode_logs/cfg1.monitor.csv
 • TensorBoard → tensorboard --logdir ./logs/tensorboard/
 • Eval Results → ./logs/eval_logs/cfg1/
 • Best Model → ./logs/best_models/cfg1/

Logging to ./logs/tensorboard/cfg1_p100_2


Output()

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 300      |
|    ep_rew_mean      | 7        |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 1        |
|    fps              | 685      |
|    time_elapsed     | 0        |
|    total_timesteps  | 300      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 449      |
|    ep_rew_mean      | 17.5     |
|    exploration_rate | 0.998    |
| time/               |          |
|    episodes         | 2        |
|    fps              | 879      |
|    time_elapsed     | 1        |
|    total_timesteps  | 898      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 411      |
|    ep_rew_mean      | 15.7     |
|    exploration_rate | 0.997    |
| time/               |          |
|    episodes       

Step 10,000 | VRAM: 0.03G alloc | 0.05G res | 0.03G peak

Eval num_timesteps=10000, episode_reward=18.80 +/- 6.24

...

Eval num_timesteps=1500000, episode_reward=7.00 +/- 3.74

Episode length: 461.60 +/- 58.29

----------------------------------
| eval/               |          |
|    mean_ep_length   | 462      |
|    mean_reward      | 7        |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 1500000  |
| train/              |          |
|    learning_rate    | 0.0002   |
|    loss             | 1.11     |
|    n_updates        | 373749   |
----------------------------------



Evaluating final policy...
 TRAINING COMPLETE
 Config ID : 1
 Mean Reward : 9.0 ± 4.5
 Peak VRAM : 0.03 GB
 Model Saved : dqn_pacman_p100_cfg1.zip
 TensorBoard : tensorboard --logdir ./logs/tensorboard/

Starting LIVE RENDER – Close window to stop...


RuntimeError: Failed to initialize SDL